In [3]:
import tensorflow as tf
import scipy.io as sio
import matplotlib.pyplot as plt
import os
import numpy as np
import shutil
from datetime import datetime
now = datetime.now()

shutil.rmtree('D:/MATLAB/OSU/OSU-automapping/Train/log/', ignore_errors=True)

sess = tf.InteractiveSession()

In [4]:
# Define functions

def conv2d(x, W, dy ,dx):
  return tf.nn.conv2d(x, W, strides=[1, dx, dy, 1], padding='SAME')

def max_pool_axb(x,a,b):
  return tf.nn.max_pool(x, ksize=[1, a, b, 1],
                        strides=[1, a, b, 1], padding='SAME')

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def variable_summaries(name, var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope(name):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

# Number of neurons , filters and layers

In [5]:
nx_conv1=32    # filter_width (frequency)
my_conv1=8   # filter_height (time)
dnx_conv1=2    # stride_width
dmy_conv1=8   # stride_height

channels_conv1=128    # channels num

nx_conv2=4
my_conv2=8
dnx_conv2=2
dmy_conv2=2

channels_conv2=128


nx_conv3=4
my_conv3=8
dnx_conv3=4
dmy_conv3=8

channels_conv3=128

size_input_fc1=int(128*128*channels_conv3/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2/dnx_conv3/dmy_conv3)
# size_input_fc1=int(128*128*channels_conv2/dnx_conv1/dmy_conv1/dnx_conv2/dmy_conv2)

num_neurons_fc1=256

print(size_input_fc1)

1024


# Network Structure

In [8]:
x = tf.placeholder("float", shape=[None, 128*128] , name='flat_input')
y_ = tf.placeholder("float", shape=[None, 4] , name='target')

x_image = tf.reshape(x, [-1,128,128,1] , name='2D_input')


W_conv1 = weight_variable([nx_conv1,my_conv1,1,channels_conv1])
b_conv1 = bias_variable([channels_conv1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1, dnx_conv1, dmy_conv1 ) + b_conv1)


W_conv2 = weight_variable([nx_conv2, my_conv2, channels_conv1, channels_conv2])
b_conv2 = bias_variable([channels_conv2])

h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, dnx_conv2, dmy_conv2 ) + b_conv2)


W_conv3 = weight_variable([nx_conv3, my_conv3, channels_conv2, channels_conv3])
b_conv3 = bias_variable([channels_conv3])

h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, dnx_conv3, dmy_conv3 ) + b_conv3)


W_fc1 = weight_variable([size_input_fc1, num_neurons_fc1])
b_fc1 = bias_variable([num_neurons_fc1])


# h_conv2_flat = tf.reshape(h_conv2, [-1, size_input_fc1])
# h_fc1 = tf.nn.tanh(tf.matmul(h_conv2_flat, W_fc1) + b_fc1)

h_conv3_flat = tf.reshape(h_conv3, [-1, size_input_fc1])
h_fc1 = tf.nn.relu(tf.matmul(h_conv3_flat, W_fc1) + b_fc1)


W_fc2 = weight_variable([num_neurons_fc1, 4])
b_fc2 = bias_variable([4])


keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)


y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)


cross_entropy_vector = -y_*tf.log(tf.clip_by_value(y_conv,1e-5,1.0))
cross_entropy = -tf.reduce_mean(y_*tf.log(y_conv))
MSE = tf.reduce_mean(tf.square(y_-y_conv))


optimizer = tf.train.AdamOptimizer(1e-4)
train_step = optimizer.minimize(MSE)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))



# Read test data and initialize the session

In [9]:
testData=sio.loadmat('C:/OSU/SongMat/Test/532522 SakiZ - osu!memories.mat')
inputTest=testData['input'].reshape(-1,128*128)
targetTest=testData['target'].reshape(-1,4)
NTest=targetTest.shape[0]

In [10]:
# tensorboard --logdir=D:\MATLAB\OSU\OSU-automapping\train\log\
sess.run(tf.global_variables_initializer())


In [11]:
saver = tf.train.Saver()


In [ ]:
step_size = 100
 
dataList=os.listdir('C:/OSU/SongMat/Train/')
acc=np.zeros([1000,1])


tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('cross_entropy', cross_entropy)
tf.summary.scalar('mse', MSE)
variable_summaries('cross_entropy_individual',cross_entropy_vector)


# tf.summary.scalar('W_conv1_max',tf.reduce_max(tf.abs(W_conv1)))
# tf.summary.scalar('b_conv1_max',tf.reduce_max(tf.abs(b_conv1)))
# tf.summary.scalar('W_conv2_max',tf.reduce_max(tf.abs(W_conv2)))
# tf.summary.scalar('b_conv2_max',tf.reduce_max(tf.abs(b_conv2)))
# tf.summary.scalar('W_conv3_max',tf.reduce_max(tf.abs(W_conv2)))
# tf.summary.scalar('b_conv3_max',tf.reduce_max(tf.abs(b_conv2)))
# tf.summary.scalar('W_fc1_max',tf.reduce_max(tf.abs(W_fc1)))
# tf.summary.scalar('b_fc1_max',tf.reduce_max(tf.abs(b_fc1)))
# tf.summary.scalar('W_fc2_max',tf.reduce_max(tf.abs(W_fc2)))
# tf.summary.scalar('b_fc2_max',tf.reduce_max(tf.abs(b_fc2)))

# variable_summaries('W_conv_1',W_conv1)
# variable_summaries('b_conv_1',b_conv1)
# variable_summaries('W_fc2',W_fc2)
# variable_summaries('b_fc2',b_fc2)


merged = tf.summary.merge_all()
train_writer = tf.summary.FileWriter('D:/MATLAB/OSU/OSU-automapping/Train/log/train'+now.strftime("%Y%m%d-%H%M%S") + "/",sess.graph)
test_writer = tf.summary.FileWriter('D:/MATLAB/OSU/OSU-automapping/Train/log/test'+now.strftime("%Y%m%d-%H%M%S") + "/")


n = 1
total_steps = 0

while n < np.size(dataList)-15:
    inputOsu=np.empty([0,128*128])
    targetOsu=np.empty([0,4])
    for k in range(n,n+15):
        
        data=sio.loadmat('C:/OSU/SongMat/Train/'+dataList[k])
        inputOsu=np.append(inputOsu,data['input1'].reshape(-1,128*128),axis=0)
        targetOsu=np.append(targetOsu,data['target1'].reshape(-1,4),axis=0)
                   
    N=inputOsu.shape[0]
    print(N)
    
    #shuffle the order
    order = np.arange(N-step_size)
    np.random.shuffle(order)
    orderAcc = np.arange(NTest)
    np.random.shuffle(orderAcc)
    
    print(targetTest[orderAcc[0:10],:])
    for i in range(N-step_size):
        if i%step_size == 0:
            summary, _  = sess.run([merged, train_step], feed_dict={x: inputOsu[order[i:i+step_size],:], y_: targetOsu[order[i:i+step_size],:], keep_prob: 0.5})
            test_writer.add_summary(summary, i + total_steps)
        else:
            summary = sess.run(merged, feed_dict={x: inputOsu[order[i:i+step_size],:], y_: targetOsu[order[i:i+step_size],:], keep_prob: 0.5})
            train_writer.add_summary(summary, i + total_steps)
        if i%1000 == 0:
            accMemory = accuracy.eval(feed_dict={x: inputTest[orderAcc[0:500],:], y_: targetTest[orderAcc[0:500],:], keep_prob: 1.0}) 
            print("step %d, memory accuracy %g" %(i, accMemory))
            print(y_conv.eval(feed_dict={x: inputTest[orderAcc[0:10],:], keep_prob: 1.0}))
            
    total_steps += i
    
    
    
    # Test accuracy
    orderAcc = np.arange(NTest)
    np.random.shuffle(orderAcc)
    accTest=accuracy.eval(feed_dict={x: inputTest[orderAcc[0:500],:], y_: targetTest[orderAcc[0:500],:], keep_prob: 1.0});
#     print("train accuracy %g"%accTrain)
    print("test accuracy %g"%accTest)
#     try:
#         print('learning_rate %g'%lr)
# #         print('learning_rate %g'%optimizer._lr)
#     except:
#         print("Error: ??")
    
    print(y_conv.eval(feed_dict={x: inputTest[orderAcc[0:10],:], keep_prob: 1.0}))
    acc[n]=accTest
    print(n)
    if acc[n]>0.8:
        break
    n=n+3
        

19184
[[0 0 0 1]
 [0 0 0 1]
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]
step 0, memory accuracy 0.102
[[  9.44954268e-33   0.00000000e+00   1.00000000e+00   4.09613489e-11]
 [  8.57691884e-22   0.00000000e+00   1.00000000e+00   8.72244949e-17]
 [  6.43126791e-15   0.00000000e+00   1.00000000e+00   3.04538973e-14]
 [  1.20895949e-19   0.00000000e+00   1.00000000e+00   1.10120085e-15]
 [  1.13633448e-07   8.32934554e-38   9.99999881e-01   2.33411290e-10]
 [  8.88903513e-22   0.00000000e+00   1.00000000e+00   7.03981469e-19]
 [  5.90450488e-21   6.31761992e-36   1.00000000e+00   1.09499562e-10]
 [  9.01207714e-24   0.00000000e+00   1.00000000e+00   7.73451839e-21]
 [  9.96819037e-25   0.00000000e+00   1.00000000e+00   1.11054591e-13]
 [  1.71952803e-29   0.00000000e+00   1.00000000e+00   1.33386339e-20]]
step 1000, memory accuracy 0.582
[[  0.00000000e+00   0.00000000e+00   4.72603117e-14   1.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   

step 15000, memory accuracy 0.582
[[ 0.25267088  0.15170346  0.16328686  0.43233874]
 [ 0.25267088  0.15170346  0.16328686  0.43233874]
 [ 0.25267088  0.15170346  0.16328686  0.43233874]
 [ 0.25267088  0.15170346  0.16328686  0.43233874]
 [ 0.25267088  0.15170346  0.16328686  0.43233874]
 [ 0.25267088  0.15170346  0.16328686  0.43233874]
 [ 0.25267088  0.15170346  0.16328686  0.43233874]
 [ 0.25267088  0.15170346  0.16328686  0.43233874]
 [ 0.25267088  0.15170346  0.16328686  0.43233874]
 [ 0.25267088  0.15170346  0.16328686  0.43233874]]
step 16000, memory accuracy 0.582
[[ 0.25450939  0.12944825  0.13145046  0.48459193]
 [ 0.25450939  0.12944825  0.13145046  0.48459193]
 [ 0.25450939  0.12944825  0.13145046  0.48459193]
 [ 0.25450939  0.12944825  0.13145046  0.48459193]
 [ 0.25450939  0.12944825  0.13145046  0.48459193]
 [ 0.25450939  0.12944825  0.13145046  0.48459193]
 [ 0.25450939  0.12944825  0.13145046  0.48459193]
 [ 0.25450939  0.12944825  0.13145046  0.48459193]
 [ 0.25450939

In [ ]:
save_path = saver.save(sess, "D:/OSU/results/save/",global_step=n)

In [ ]:
print(targetOsu[120,:])
plt.pcolor(inputOsu[120,:].reshape([128,128]))
plt.colorbar()
plt.show()

In [ ]:
targetOsu2[2015:2020,:]

In [ ]:
print(acc)

In [ ]:
print(y_conv.eval(feed_dict={x: inputOsu[200:210:1,:], keep_prob: 1.0}))
print ("test accuracy %g"%accuracy.eval(feed_dict={
  x: inputOsu[-500:,:], y_: targetOsu2[-500:,:], keep_prob: 1.0}))

In [ ]:
# Save the output of new maps 

dataList=os.listdir('C:/OSU/SongMat/Create/')
# inputOsuNew=np.empty([0,128*128]);
for i in range(np.size(dataList)):
    if not dataList[i][0]=='y':
        data=sio.loadmat('C:/OSU/SongMat/Create/'+dataList[i])
        print(data['osuFolder'])
        
        inputOsuNew=data['input'].reshape(-1,128*128)
        outputOsuNew=data['target']
        outputOsu2New=np.zeros([outputOsuNew.shape[0],2]);
        outputOsu2New[:,0]=outputOsuNew[:,0]+outputOsuNew[:,1]+outputOsuNew[:,2]
        outputOsu2New[:,1]=outputOsuNew[:,3]
        
        y_result=np.empty([0,2])
        
        step_size=500
        cursor=0
        print ("test accuracy %g"%accuracy.eval(feed_dict={x: inputOsuNew[-500:,:], y_: outputOsu2New[-500:,:], keep_prob: 1.0}))
        while cursor<inputOsuNew.shape[0]:
            if inputOsuNew[cursor:,:].shape[0]>step_size:
                y_result1=y_conv.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            else:
                y_result1=y_conv.eval(feed_dict={x: inputOsuNew[cursor:(cursor+step_size):1,:], keep_prob: 1.0})
                y_result=np.append(y_result,y_result1,axis=0)
            cursor+=step_size
            
        data_save={'y2':y_result}
        sio.savemat('C:/OSU/SongMat/Create/y2_'+dataList[i],data_save)


In [ ]:
data_save={'y':y_result}
sio.savemat('D:/OSU/results/innocence/y_innocence.mat',data_save)

In [ ]:
checkpoint_dir='D:/OSU/results/innocence/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
saver2 = tf.train.Saver()

In [ ]:
checkpoint_dir='/Users/dongqihan/Desktop/FrOstNova/save/'
ckpt = tf.train.get_checkpoint_state(checkpoint_dir)  
saver2.restore(sess, ckpt.model_checkpoint_path)  

In [ ]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess_tmp = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess_tmp.run(c))

In [ ]:
data_test=sio.loadmat('/Users/dongqihan/Desktop/FrOstNova/granat.mat')
inputOsu_test=data_test['input'].reshape(-1,128*128)
y_result_test=y_conv.eval(feed_dict={x: inputOsu_test, keep_prob: 1.0})
data={'y':y_result_test}
sio.savemat('/Users/dongqihan/Desktop/FrOstNova/y_memory2granat.mat',data)